Implement PKCS#7 padding
A block cipher transforms a fixed-sized block (usually 8 or 16 bytes) of plaintext into ciphertext. But we almost never want to transform a single block; we encrypt irregularly-sized messages.

One way we account for irregularly-sized messages is by padding, creating a plaintext that is an even multiple of the blocksize. The most popular padding scheme is called PKCS#7.

So: pad any block to a specific block length, by appending the number of bytes of padding to the end of the block. For instance,

"YELLOW SUBMARINE"
... padded to 20 bytes would be:

"YELLOW SUBMARINE\x04\x04\x04\x04"

In [7]:
def PKCS7(text, blockLength):
    padSize = blockLength - (len(text) % blockLength)
    return bytes(text + '\x04' * padSize, 'utf-8')

plaintext = 'YELLOW SUBMARINE'
padded = PKCS7(plaintext, 20)
print(padded)

b'YELLOW SUBMARINE\x04\x04\x04\x04'


Implement CBC mode
CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

The file here is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)

In [25]:
import base64
from Crypto.Cipher import AES
def AESdecrypt(key, ciphertext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.decrypt(ciphertext)

def AESencrypt(key, ciphertext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(ciphertext)

def XOR(byteString1, byteString2):
    result = b''
    for b1,b2 in zip(byteString1, byteString2):
        result += bytes([b1 ^ b2])
    return result

def CBCdecrypt(key, ciphertext, IV):
    blockSize = 16
    plaintext = b''
    cipherBlocks = [ciphertext[i:i + blockSize] for i in range(0,len(ciphertext), blockSize)]
    for i in range(len(cipherBlocks) - 1, -1, -1):
        decryptedBlock = AESdecrypt(key, cipherBlocks[i])
        previousBlock = IV if i == 0 else cipherBlocks[i-1]
        plaintext = XOR(previousBlock, decryptedBlock) + plaintext
    return plaintext

def CBCencrypt(key, plaintext, IV):
    blockSize = 16
    ciphertext = b''
    plaintextBlocks = [plaintext[i:i + blockSize] for i in range(0,len(plaintext), blockSize)]
    prevCipherBlock = IV
    for i in range(len(plaintextBlocks)):
        cipherBlock = AESencrypt(key, XOR(prevCipherBlock, plaintextBlocks[i]))
        prevCipherBlock = cipherBlock
        ciphertext += cipherBlock
    return ciphertext

key = b'YELLOW SUBMARINE'
IV = b'\x00' * 16
blockSize = 16

with open('set2_chall10.txt','r') as file:
    base64d = file.read().replace('\n','')
    ciphertext = base64.b64decode(base64d)
    plaintext = CBCdecrypt(key, ciphertext, IV)
    print(plaintext)
plaintext = b'hello world' + b'\x04' * 5
ciphertext = CBCencrypt(key, plaintext, IV)
print(ciphertext)
plaintext = CBCdecrypt(key, ciphertext, IV)
print(plaintext)



b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

An ECB/CBC detection oracle
Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:

encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.